In [61]:
#importing packages

In [62]:
import pandas as pd
import sqlite3



In [63]:
#Connecting Jupyter notebook to database file.

In [64]:
%%capture
%load_ext sql
%sql sqlite:///gamesalesdata5.db



In [65]:
#Connecting Jupyter notebook to database file.

In [66]:
%%sql

SELECT *
  FROM sqlite_master
 WHERE type='table'; 

 * sqlite:///gamesalesdata5.db
Done.


type,name,tbl_name,rootpage,sql
table,gamesalesproject1,gamesalesproject1,2,"CREATE TABLE ""gamesalesproject1"" ( ""game"" TEXT, ""platform"" TEXT, ""publisher"" TEXT, ""developer"" TEXT, ""games_sold"" REAL, ""year"" INTEGER)"
table,gamereviewsdata,gamereviewsdata,358,"CREATE TABLE ""gamereviewsdata"" ( ""game"" TEXT, ""critic_score"" REAL, ""user_score"" REAL)"


The database contains two tables: **_gamesalesproject1_**  and **_gamesreviewsdata_** . There are over 19,000 rows in this dataset. Data source is Kaggle. 

**_gamesalesproject1_** has 6 columns:

**game** (The name of the video game)

**platform** (Gaming Platform)

**publisher** (Game Publisher)

**developer** (Game Developer)

**games_sold** (Number of games sold in millions)

**year** (release year)


<br> <br>

**_gamesreviewsdata_** has 3 columns:

**game**(The name of the video game)

**critic_score** (Critic score according to Metacritic)

**user_score** (User score according to Metacritic)

In this project I'll use SQL to explore and analyze the data set. Some concepts used will be JOINS, set theories such as INTERCEPT, common table expressions, and subqueries. 

In [67]:
# The %%sql line allows you to run SQL queries in Jupyter Notebook

# Select all information for the top ten best-selling games
# Order the results from best-selling game down to tenth best-selling

In [68]:
%%sql  

SELECT *
FROM gamesalesproject1
ORDER BY games_sold DESC
LIMIT 10;

 * sqlite:///gamesalesdata5.db
Done.


game,platform,publisher,developer,games_sold,year
Wii Sports,Wii,Nintendo,Nintendo EAD,82.9,2006
Super Mario Bros.,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,40.0,2012
Mario Kart Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,36.6,2017
Minecraft,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros.,DS,Nintendo,Nintendo EAD,30.8,2006
New Super Mario Bros. Wii,Wii,Nintendo,Nintendo EAD,30.3,2009


In [69]:
# Join the two tables and find the number of games where there
# are no reviews data scores.

In [70]:
%%sql 


SELECT COUNT(g.game)
FROM gamesalesproject1 as g
LEFT JOIN gamereviewsdata as r
ON g.game = r.game
WHERE critic_score IS NULL and user_score IS NULL; 



 * sqlite:///gamesalesdata5.db
Done.


COUNT(g.game)
17248


In [71]:
# Select release year and average critic score for each year, rounded and aliased
# Join the game_sales and reviews tables
# Group by release year
# Order the data from highest to lowest avg_critic_score and limit to 10 results

In [72]:
%%sql

SELECT g.year, ROUND(AVG(r.critic_score),2) AS avg_critic_score
FROM gamesalesproject1 as g
LEFT JOIN gamereviewsdata as r 
ON g.game = r.game
GROUP BY g.year
ORDER BY avg_critic_score DESC 
LIMIT 10;

 * sqlite:///gamesalesdata5.db
Done.


year,avg_critic_score
1984,9.5
1992,8.68
1990,8.54
1991,8.32
2020,8.26
1994,8.04
2019,7.97
1985,7.84
1993,7.72
2013,7.61


In [73]:
# 1984 looks suspiciously high. Update the query to add a count of games released in each year called num_games
# Update the query so that it only returns years that have more than four reviewed games

In [74]:
%%sql

SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.critic_score),2) AS avg_critic_score
FROM gamesalesproject1 as g
INNER JOIN gamereviewsdata as r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.game) > 4
ORDER BY avg_critic_score DESC
LIMIT 10;

 * sqlite:///gamesalesdata5.db
Done.


year,num_games,avg_critic_score
1984,12,9.5
1992,56,8.68
1990,30,8.54
1991,39,8.32
2020,13,8.26
1994,131,8.04
2019,95,7.97
1985,19,7.84
1993,94,7.72
2013,1631,7.61


In [75]:
# Time to compare with user scores. Select year, an average of user_score, and a count of games released in a given year, aliased and rounded
# Include only years with more than four reviewed games; group data by year
# Order data by avg_user_score, and limit to ten results

In [76]:
%%sql

SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.user_score),2) as avg_user_score
FROM gamesalesproject1 as g 
INNER JOIN gamereviewsdata as r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.year) > 4
ORDER BY avg_user_score DESC
LIMIT 10;


 * sqlite:///gamesalesdata5.db
Done.


year,num_games,avg_user_score
1997,333,9.38
1990,30,9.3
1999,478,9.1
2003,1695,8.88
1998,423,8.86
1994,131,8.8
1987,18,8.8
2007,2779,8.64
2000,545,8.52
1996,351,8.4


In [77]:
# Select the year results that appear on both critics and users 
# from the above queries

In [78]:
%%sql

WITH critics as (SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.critic_score),2) AS avg_critic_score
FROM gamesalesproject1 as g
INNER JOIN gamereviewsdata as r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.game) > 4
ORDER BY avg_critic_score DESC
LIMIT 10),

users as (SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.user_score),2) as avg_user_score
FROM gamesalesproject1 as g 
INNER JOIN gamereviewsdata as r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.year) > 4
ORDER BY avg_user_score DESC
LIMIT 10)

SELECT year
FROM critics
INTERSECT
SELECT year 
FROM users;

 * sqlite:///gamesalesdata5.db
Done.


year
1990
1994


In [79]:
# Early 1990's appears to a golden age for video games. Select year and sum of games_sold, aliased as total_games_sold; order results by total_games_sold descending
# Filter game_sales based on whether each year is in the list returned in the previous task

In [80]:
%%sql 

WITH critics as (SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.critic_score),2) AS avg_critic_score
FROM gamesalesproject1 as g
INNER JOIN gamereviewsdata as r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.game) > 4
ORDER BY avg_critic_score DESC
LIMIT 10),

users as (SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.user_score),2) as avg_user_score
FROM gamesalesproject1 as g 
INNER JOIN gamereviewsdata as r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.year) > 4
ORDER BY avg_user_score DESC
LIMIT 10)

SELECT g.year, SUM(g.games_sold) AS total_games_sold
FROM gamesalesproject1 as g 
WHERE g.year in (SELECT year FROM users
   INTERSECT
SELECT year FROM critics)
GROUP BY g.year
ORDER BY total_games_sold DESC;



 * sqlite:///gamesalesdata5.db
Done.


year,total_games_sold
1994,62.969999999999985
1990,52.4
